# 01_convert_bottom_to_top.ipynb


## Aufgabe

In diesem Notebook soll die Positionierung von HTML-Elementen, die per `bottom` definiert sind, in `top`-Positionen umgerechnet werden.

Ziel: JasperReport benötigt `top`, daher wird `bottom: Xpx` in `top: (HTML-Höhe - X + offset_y)` umgerechnet.


## Bedienung

1. HTML-Code kann eingegeben werden:
   - als direkter HTML-String (in Textfeld)
   - oder per Dateipfad aus `data/original/`
2. Offset-Werte für X und Y optional einstellbar
3. Umrechnung per Button starten
4. Ausgabe wird angezeigt und in `data/output/` gespeichert


## Technische Konstanten

- HTML_WIDTH = 1210
- HTML_HEIGHT = 825
- HTML_MARGIN_TOP = 0
- HTML_MARGIN_BOTTOM = 0


In [ ]:
# Benötigte Bibliotheken importieren
import sys
import os
import datetime

# Pfad zum shared-Verzeichnis hinzufügen, falls es nicht im Pythonpath ist
sys.path.append('..')

# Importieren der gemeinsam genutzten Funktionen
from shared.html_utils import convert_bottom_to_top, load_html_from_file, save_html_to_file, save_original_html
from shared.constants import HTML_HEIGHT


In [ ]:
# Installieren von benötigten Paketen, falls nicht vorhanden
try:
    import bs4
    print(f"BeautifulSoup Version: {bs4.__version__}")
except ImportError:
    print("BeautifulSoup ist nicht installiert. Installation wird gestartet...")
    !pip install beautifulsoup4
    import bs4
    print(f"BeautifulSoup Version: {bs4.__version__}")

try:
    import ipywidgets as widgets
    from IPython.display import display, HTML
    print(f"ipywidgets Version: {widgets.__version__}")
except ImportError:
    print("ipywidgets ist nicht installiert. Installation wird gestartet...")
    !pip install ipywidgets
    import ipywidgets as widgets
    from IPython.display import display, HTML
    print(f"ipywidgets Version: {widgets.__version__}")


In [ ]:
# Beispiel-HTML für die Demonstration
example_html = """<!DOCTYPE html>
<html>
<head>
    <meta http-equiv="X-UA-Compatible" content="IE=Edge" />
    <meta charset="utf-8" />
</head>

<body style="margin: 0;">

<div id="p1" style="overflow: hidden; position: relative; background-color: white; width: 1210px; height: 825px;">

    <!-- Begin shared CSS values -->
    <style class="shared-css" type="text/css" >
        .t {
            transform-origin: bottom left;
            z-index: 2;
            position: absolute;
            white-space: pre;
            overflow: visible;
            line-height: 1.5;
        }
        .text-container {
            white-space: pre;
        }
        @supports (-webkit-touch-callout: none) {
            .text-container {
                white-space: normal;
            }
        }
    </style>
    <!-- End shared CSS values -->

    <!-- Begin inline CSS -->
    <style type="text/css" >
        #t1_1{left:18px;bottom:804px;letter-spacing:0.14px;}
        #t2_1{left:128px;bottom:804px;}
        #t3_1{left:165px;bottom:804px;letter-spacing:0.15px;}
    </style>
    <!-- End inline CSS -->

    <!-- Begin text -->
    <div id="t1_1" class="t">Beispieltext 1</div>
    <div id="t2_1" class="t">Beispieltext 2</div>
    <div id="t3_1" class="t">Beispieltext 3</div>
    <!-- End text -->

</div>
</body>
</html>"""


In [ ]:
# Erstellen der Widgets für die Benutzeroberfläche
html_input = widgets.Textarea(
    value=example_html,
    placeholder='HTML-Code hier einfügen',
    description='HTML-Code:',
    disabled=False,
    layout=widgets.Layout(width='100%', height='200px')
)

file_input = widgets.Text(
    value='',
    placeholder='Dateipfad eingeben (z.B. original_2025-07-16_104156.html)',
    description='Datei:',
    disabled=False,
    layout=widgets.Layout(width='80%')
)

offset_x = widgets.IntSlider(
    value=0,
    min=-50,
    max=50,
    step=1,
    description='Offset X:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)

offset_y = widgets.IntSlider(
    value=0,
    min=-50,
    max=50,
    step=1,
    description='Offset Y:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)

convert_button = widgets.Button(
    description='Umrechnen',
    disabled=False,
    button_style='success',
    tooltip='Klicken, um bottom zu top umzurechnen',
    icon='check'
)

output = widgets.Output()

# Anzeigen der Widgets
print("HTML-Code eingeben oder Datei auswählen:")
display(html_input)
print("\nODER Datei aus data/original/ auswählen:")
display(file_input)
print("\nOffset-Werte einstellen (optional):")
display(offset_x)
display(offset_y)
display(convert_button)
display(output)


In [ ]:
# Funktion für den Konvertierungsbutton
def on_convert_button_clicked(b):
    with output:
        output.clear_output()
        
        # HTML-Code aus Textfeld oder Datei laden
        html_string = ""
        if file_input.value:
            file_path = os.path.join('..', 'data', 'original', file_input.value)
            print(f"Lade HTML aus Datei: {file_path}")
            html_string = load_html_from_file(file_path)
            if not html_string:
                print("Fehler: Datei konnte nicht geladen werden.")
                return
        else:
            html_string = html_input.value
            if not html_string.strip():
                print("Fehler: Kein HTML-Code eingegeben.")
                return
            
            # Original-HTML speichern
            timestamp = datetime.datetime.now().strftime("%Y-%m-%d_%H%M%S")
            original_path = save_original_html(html_string, timestamp)
            print(f"Original-HTML gespeichert unter: {original_path}")
        
        # Offset-Werte auslesen
        offset_x_value = offset_x.value
        offset_y_value = offset_y.value
        
        print(f"Konvertiere mit Offset X: {offset_x_value}, Offset Y: {offset_y_value}")
        
        # Konvertierung durchführen
        converted_html = convert_bottom_to_top(html_string, offset_x_value, offset_y_value)
        
        # Konvertiertes HTML speichern
        timestamp = datetime.datetime.now().strftime("%Y-%m-%d_%H%M%S")
        function_name = "convert_bottom_to_top"
        if offset_x_value != 0 or offset_y_value != 0:
            function_name += f"_with_offset_x{offset_x_value}_y{offset_y_value}"
        output_path = save_html_to_file(converted_html, function_name, timestamp)
        
        print(f"Konvertiertes HTML gespeichert unter: {output_path}")
        print("\nErgebnis (Ausschnitt):")
        print(converted_html[:500] + "...")
        
        # HTML-Vorschau anzeigen
        print("\nHTML-Vorschau:")
        display(HTML(converted_html))

# Button-Klick-Event registrieren
convert_button.on_click(on_convert_button_clicked)


## Erklärung der Umrechnung

Die Umrechnung von `bottom` zu `top` erfolgt nach folgender Formel:

```
top = HTML_HEIGHT - bottom + offset_y
```

Dabei ist:
- `HTML_HEIGHT`: Die Höhe des HTML-Dokuments (825px)
- `bottom`: Der Wert der bottom-Eigenschaft in Pixeln
- `offset_y`: Ein optionaler vertikaler Offset (positiv = nach unten, negativ = nach oben)

Zusätzlich kann ein horizontaler Offset (`offset_x`) auf die `left`-Werte angewendet werden:

```
left = left + offset_x
```